# Main.py

In [ ]:
import datetime
from tkinter import Tk, Entry, Text, LEFT, RIGHT, END
import itertools
import sys
import deadline as dl
import taskmanager as tm

class GUI:

    def __init__(self, task_manager):
        self.task_manager = task_manager
        self.window = Tk()
        self.window.geometry('800x700')  # set Window size
        self.window.title('Task Terminator T800')  # set Window title

        self.input_box = Entry(self.window)  # create an input box
        self.input_box.pack(padx=5, pady=5, fill='x')  # make the input box fill the width of the Window
        self.input_box.bind('<Return>', self.command_entered)  # bind the command_entered function to the Enter key
        self.input_box.focus()  # set focus to the input box

        # add a text area to show the chat history
        self.history_area = Text(self.window, width="50")
        self.history_area.pack(padx=5, pady=5, side=LEFT, fill="y")
        self.output_font = ('Courier New', 11)
        self.history_area.tag_configure('error_format', foreground='red', font=self.output_font)
        self.history_area.tag_configure('success_format', foreground='green', font=self.output_font)
        self.history_area.tag_configure('normal_format', font=self.output_font)

        # add a text area to show the list of tasks
        self.list_area = Text(self.window)
        self.list_area.pack(padx=5, pady=5, side=RIGHT, fill="both")
        self.list_area.tag_configure('normal_format',  font=self.output_font)
        self.list_area.tag_configure('pending_format', foreground='red', font=self.output_font)
        self.list_area.tag_configure('done_format', foreground='green', font=self.output_font)

        # show the welcome message and the list of tasks
        self.update_chat_history('start', 'Welcome to T800!', 'success_format')
        self.update_task_list(self.task_manager.items)

    def update_chat_history(self, command, response, status_format):
        """
        status_format: indicates which color to use for the status message
          can be 'error_format', 'success_format', or 'normal_format'
        """
        current_time = datetime.datetime.now().strftime("%H:%M:%S")
        self.history_area.insert(1.0, '-' * 40 + '\n', 'normal_format')
        self.history_area.insert(1.0, '>>> ' + str(response) + '\n', status_format)
        self.history_area.insert(1.0, 'You said: ' + str(command) + '\n', 'normal_format')
        self.history_area.insert(1.0, current_time + '\n', 'normal_format')
    
    def update_task_list(self, tasks):
        self.list_area.delete('1.0', END)  # clear the list area
        self.list_area.insert(END, """           ______         __                  
          /_  __/__ ____ / /__                
           / / / _ `(_-</  '_/                
  ______  /_/  \_,_/___/_/\_\     __          
 /_  __/__ ______ _  (_)__  ___ _/ /____  ____
  / / / -_) __/  ' \/ / _ \/ _ `/ __/ _ \/ __/
 /_/  \__/_/ /_/_/_/_/_//_/\_,_/\__/\___/_/   
\n""")
        self.list_area.insert(END, """==============================================
STATUS | INDEX | DESCRIPTION      | DEADLINE
----------------------------------------------\n""")
        if len(tasks) == 0:
            output_format = 'normal_format'
            self.list_area.insert(END, '>>> Nothing to list', output_format)
        else:
            output_format = 'normal_format'
            
            for i, task in enumerate(tasks):
                output_format = 'done_format' if task.is_done else 'pending_format'
                
                deadline_arr = []
                desc_arr = []
                
                if isinstance(task, dl.Deadline):
                    to_print = str(task)[:6] + '|' + str(i+1).center(6) + '| ' + str(task)[6:20] + \
                    ' | ' + str(task.by)[:8] + '\n'
                    self.list_area.insert(END, to_print, output_format) # print first 8 chars of 'deadline' only
                    
                    if len(str(task.by)) > 8:
                        deadline_arr = self.__string_splitter(deadline_arr, task.by, 8) [1:] # if longer than 8 char, split string
                else:
                    to_print = str(task)[:6] + '|' + str(i+1).center(6) + '| ' + str(task)[6:20] + \
                    ' | ' + '-' + '\n'
                    self.list_area.insert(END, to_print, output_format) # print first 14 chars of 'deadline' only
            
                if len(str(task.description)) > 14:
                    desc_arr = self.__string_splitter(desc_arr, task.description, 14)[1:] # if longer than 14 char, split string

                for combination in itertools.zip_longest(desc_arr, deadline_arr, fillvalue=""):                    
                    to_print = " "*15 + combination[0].ljust(14) + " "*3 + combination[1].ljust(8)  + '\n'
                    self.list_area.insert(END, to_print, output_format) # iterate thru 2 lists simultanously, filling up diff with ""
            self.list_area.insert(END, """----------------------------------------------\n""")
    
    def __string_splitter(self, arr, string, split_length):
        """ Recursively splits the string greedily in length specified by split_length"""
        if len(string) < split_length:
            arr.append(string)
            return arr
        else:
            arr.append(string[:split_length])
            return self.__string_splitter(arr, string[split_length:], split_length)
    
    def clear_input_box(self):
        self.input_box.delete(0, END)
    
    def command_entered(self, event):
        command = None
        try:
            command = self.input_box.get().strip()
            if command.lower() == 'exit':
                self.window.destroy()
                sys.exit()
            output = self.task_manager.execute_command(command)
            self.update_chat_history(command, output, 'success_format')
            self.update_task_list(self.task_manager.items)
            self.clear_input_box()
            self.task_manager.save_data()
        except Exception as e:
            self.update_chat_history(command, str(e), 'error_format')

    def start(self):
        self.window.mainloop()
        
if __name__ == '__main__' :
    GUI(tm.TaskManager()).start()

# TaskManager.py

In [ ]:
import re
import deadline as dl
import todo as td
import exceptions as ex
import storagemanager as sm

FILENAME = 'monty7.csv'

class TaskManager:
    """
    TaskManager class will execute task-related commands
    """
    FILENAME = 'monty7.csv'
    items = []
    def __init__(self):
        self.storage = sm.StorageManager(FILENAME)
        self.storage.load_data(self.items)
    def save_data(self):
        self.storage.save_data(self.items)
    def get_help(self):
        """
        Generates the list of commands available
        for T800.

        Returns
        -------
        None.

        """
        return """========================================
T800 can understand the following commands:
(Case-insensitive)

  todo DESCRIPTION 
    Adds a task to the list
    Example: todo read book
  deadline DESCRIPTION "by:" DEADLINE
    Adds a task with deadline to the list
    Example: deadline read book by: Tuesday
  done INDEX
    Marks the task at INDEX as 'done'
    Example: done 1
  delete INDEX
    Deletes item at the index
    Example: delete 1
  pending INDEX
    Reverts done item at index to 'pending'
    Example: pending 1
  exit
    Exits the application
  help
    Shows the help information
  progress
    Shows the progress of the current tasks
  mass TASK INDEX + **More if needed
    Performs either done, pending or delete
    of tasks simultaneously. 
    Space is required in between each task.
    Example: pending 1 3 4
============================================"""
    
    def add_item(self, user_input):
        """
        Adds a 'ToDo' type item into the local list.

        Parameters
        ----------
        user_input:
            All input after keyword 'todo' is considered valid.

        Raises
        ------
        IndexError
            If user provides blank input

        Returns
        -------
        Information on task added.

        """
        command_parts = user_input.strip().split(' ', 1)
        try:
            self.items.append(td.ToDo(command_parts[1], False))
            return ("New item: " + "'" + command_parts[1] + "'" + " added")
        except IndexError as ie:
            raise IndexError("INPUT: todo \"task\"") from ie

            
    def add_deadline_item(self, user_input):
        """
        Adds a 'Deadline' type item into the list.

        Parameters
        ----------
        user_input : Requires a keyword 'by:'
            All input after keyword 'deadline' is considered valid.

        Raises
        ------
        ex.InvalidDeadlineInput:
            When 'by' keyword is not present in input (case insensitive)
            
        ex.NoTaskError:
            When no task is provided but 'by:' keyword is present
        
        ex.NoDueDateError:
            When no due date is provided but 'by:' keyword is present
            
        ex.NoDueNoTaskError:
            When both task & due date is not provided but keyword is present

        Returns
        -------
        Information on task added.

        """
        if re.search(" by:", user_input, re.IGNORECASE):
            command_parts = user_input.strip().split(' ', 1)
            arr = re.split("by:", command_parts[1], 1, re.IGNORECASE)
            (task, due) = [x.strip() for x in arr]
            
            if not due and not task:
                raise ex.NoDueNoTaskError("INPUT: deadline \"task\" by: \"due date\"")
            elif not due:
                raise ex.NoDueDateError("No due date provided! INPUT: deadline \"task\" by: \"due date\"" )
            elif not task:
                raise ex.NoTaskError("Provide a task! INPUT: deadline \"task\" by: \"due date\"")
    
            self.items.append(dl.Deadline(task, False, due))
            return ("New item: " + "'" + task + "'" + " added. " + "Deadline: " + "'" + due + "'")
        else:
            raise ex.InvalidDeadlineInput("Missing 'by' keyword! INPUT: deadline \"task\" by: \"due date\"")
            
    def mark_item_as_done(self, user_input):
        """
        Marks a specified item as 'done'.

        Parameters
        ----------
        user_input : 
            A number within bounds of the current tasklist.

        Returns
        -------
        Information on task marked 'done' or if task has been mark 'done' prior.

        """
        
        index_as_string = user_input[5:].strip()
        index_to_remove = self.__index_check(index_as_string)
        for i, obj in enumerate(self.items):
            if i == index_to_remove:
                if obj.is_done:
                    return ("Item: " + "'" + obj.description + "'" + " has been done already")
                obj.mark_as_done()
                return ("Item: " + "'" + obj.description + "'" + " marked as done")
            
    def mark_item_as_pending(self, user_input):
        """
        Marks a specified item as 'pending'.

        Parameters
        ----------
        user_input : 
            A number within bounds of the current tasklist.

        Returns
        -------
        Information on task marked 'pending' or if task has been mark 'pending' prior.

        """
        index_as_string = user_input[8:].strip()
        index_to_remove = self.__index_check(index_as_string)
        for i, obj in enumerate(self.items):
            if i == index_to_remove:
                if not obj.is_done:
                    return ("Item: " + "'" + obj.description + "'" + " is already pending")
                obj.mark_as_pending()
                return ("Item: " + "'" + obj.description + "'" + " marked as pending")
    
    def __index_check(self, string):
        """
        Helper function to check for valid input for mark_as_done and
        mark_as_pending methods.

        Parameters
        ----------
        string : 
            A single number that was obtained from input of the mark_as_done
            or mark_as_pending functions

        Raises
        ------
        ValueError
            Non-numerical input.
        ex.ZeroInputError:
            Invalid element '0' specified
        IndexError
            No element at specified index.

        Returns
        -------
        Provided number - 1

        """
        try:
            index = int(string.strip())
        except ValueError as ve:
            raise ValueError('"{}" is not a number'.format(string)) from ve
        if index < 1:
            raise ex.ZeroInputError('Index must be greater than 0')
        try:
            if self.items[index - 1]:
                return index - 1
        except IndexError as ie:
            raise IndexError('No item at index: {}'.format(string)) from ie
            
            
    def delete_item(self, user_input):
        """
        Deletes item from local tasklist.

        Parameters
        ----------
        user_input :
            A number within bounds of the current tasklist.

        Raises
        ------
        IndexError
            No element at specified index.
        ValueError
            Non-numerical input.

        Returns
        -------
        Information on task deleted.

        """
        try:
            delete_string = int(user_input[7:].strip())
            if delete_string == 0:
                raise IndexError
            deleted_item = self.items.pop(delete_string-1)
            return ("Task: " + "'" + deleted_item.description + "'" + " deleted from the list")
        except IndexError as ie:
            raise IndexError("There is no list item at the number you typed!") from ie
        except ValueError as ve:
            raise ValueError("Only integers accepted as input") from ve
            
    def get_current_progress(self):
        """
        Obtains progress of current session, ie.
        how many ToDos and Deadlines tasks marked as done.
        Tasks marked pending after being marked done will be
        taken into account.

        Returns
        -------
        Progress for this session.

        """
        status = {'Todo': 0, 'Deadline': 0}
        status['Todo'] = td.ToDo.progress_check()
        status['Deadline'] = dl.Deadline.progress_check()
        return("""Progress for this session:
    | ToDos: {} | Deadlines: {} |""".format(status['Todo'], status['Deadline']))
    
    def mass(self, user_input):
        """
        Executes mass execution of delete, pending or done sub methods.

        Parameters
        ----------
        user_input :
            Requires either 'delete', 'done', or 'pending' to be specified.

        Raises
        ------
        ex.InvalidMassInputError
            When action keywords 'delete', 'pending', or 'done' is not specified

        Returns
        -------
        A internal call to private method mass_execute.

        """
        command = user_input[5:].strip()
        if re.search("\Adelete", command[:6], re.IGNORECASE):
            return self.__mass_execute(command.lower(), 7)
        elif re.search("\Adone", command[:4], re.IGNORECASE):
            return self.__mass_execute(command.lower(), 5)
        elif re.search("\Apending", command[:7], re.IGNORECASE):
            return self.__mass_execute(command.lower(), 8)
        else:
            raise ex.InvalidMassInputError("INPUT: 'mass' + command + args**")
            
    def __mass_execute(self, command, strip_len):
        """
        Executes 'command' determined by strip_len, sequentially.
        Will execute all numbers regardless of whether input contains non-numerical
        input or not. 
        
        E.g. 'mass done 1 e3 app0e 2' results in task 1 and 2 marked as done.

        Parameters
        ----------
        command : Number
            Numbers to execute operations on
        function_name : method
            String representing method to execute
        strip_len : Number
            Length of string to strip()

        Raises
        ------
        ex.InvalidMassInputError
            if no numbers are supplied BUT keyword is present, e.g. 
            'mass delete abce1de'

        Returns
        -------
        Information on task and what was executed on.

        """
        string = command[strip_len:]
        back_string = command[:strip_len]
        str_arr = [int(s) for s in string.split() if s.isdigit()]
        str_arr.sort(reverse=True)
        for i in str_arr:
            self.execute_command(back_string + str(i))
        if str_arr:
            return ("Executed mass action '{}' on items {}!".format(back_string.strip(), str_arr))
        else:
            raise ex.InvalidMassInputError("Nothing was done! Check your input.")
        
    def execute_command(self, command):
        """
        Main function to execute commands. All commmands are case-insensitive,
        i.e. 'DeAdlIne read book BY: 2pm' will add a Deadline item into list
        with a deadline of 2pm.

        Parameters
        ----------
        command : String
            User input from GUI box using Tkinter

        Raises
        ------
        Exception
            If command entered is not valid.

        Returns
        -------
        A call to associated command to execute with keyword parameters.

        """
        if re.search("\Ahelp", command[:4], re.IGNORECASE):
            return self.get_help()
        elif re.search("\Aprogress", command[:8], re.IGNORECASE):
            return self.get_current_progress()
        elif re.search("\Atodo", command[:4], re.IGNORECASE):
            return self.add_item(command)
        elif re.search("\Adeadline", command[:8], re.IGNORECASE):
            return self.add_deadline_item(command)
        elif re.search("\Adone", command[:4], re.IGNORECASE):
            return self.mark_item_as_done(command)
        elif re.search("\Apending", command[:7], re.IGNORECASE):
            return self.mark_item_as_pending(command)
        elif re.search("\Adelete", command[:6], re.IGNORECASE):
            return self.delete_item(command)
        elif re.search("\Amass", command[:4], re.IGNORECASE):
            return self.mass(command)
        else:
            raise Exception('Command not recognized. Input \'help\' to see all available commands.')                 

# todo.py

In [6]:
class ToDo:
    
    _progress = 0 # class-level variable
    TYPE_KEY = 'T'

    def __init__(self, description, status):
        self.description = description
        self.is_done = status
        
    def __str__(self):        
        return self.__status_as_icon().center(6) + self.description.ljust(14)

    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            ToDo._progress = ToDo._progress + 1
        self.is_done = True

    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if ToDo._progress != 0:
                ToDo._progress = ToDo._progress - 1
        self.is_done = False

    def __status_as_icon(self):
        return 'X' if self.is_done else '-'
    
    def as_csv(self):
        """ Return the details of todo object as a list,
        suitable to be stored in a csv file.
        """
        return [self.TYPE_KEY, self.description, 'done' if self.is_done else 'pending']
    
    @classmethod
    def progress_check(cls):
        return cls._progress

# deadline.py

In [ ]:
import todo as td

class Deadline (td.ToDo):
    
    _progress = 0 # class-level variable
    TYPE_KEY = 'D'

    def __init__(self, description, status, by):
        super().__init__(description, status)
        self.by = by        
    
    def __str__(self):
        s = super().__str__()
        return s[:-1] + " " + self.by
        
    def as_csv(self):
        c = super().as_csv()
        c.append(self.by)
        return c
        
    def mark_as_done(self):
        if not self.is_done: # increment progress if needed
            Deadline._progress = Deadline._progress + 1
        self.is_done = True
        
    def mark_as_pending(self):
        if self.is_done: # decrement progress if needed
            if Deadline._progress != 0:
                Deadline._progress = Deadline._progress - 1
        self.is_done = False

# storagemanager.py

In [ ]:
# -*- coding: utf-8 -*-
import csv
import deadline as dl
import todo as td

class StorageManager:
    
    def __init__(self, filename):
        self.filename = filename
        
    def load_data(self, items):
        """
        Loads data from csv specified.
        
        If csv does not exist, a new CSV file is created
        in the same directory as main. 
        
        Items are loaded from CSV into the item list.

        Returns
        -------
        None.

        """
        self.__create_file_if_missing(self.filename)
        with open(self.filename, 'r') as csvfile:
            file_handler = csv.reader(csvfile)
            for row in file_handler:
                if not row:
                    continue
                self.__load_item_from_csv_line(row, items)
            return
        
    def __create_file_if_missing(self, filename):
        """
        Creates a file. 
        
        If the named csv file is locked for editing, a permission error is
        raised to console.

        Returns
        -------
        None.

        """
        try:
            open(filename, 'a').close()
        except PermissionError as pe:
            raise PermissionError("Error creating file, check permissions.") from pe
        
        
    def __load_item_from_csv_line(self, row, items):
        """
        From the specified CSV, it will read the CSV row by row and create todo
        and deadline objects respectively.

        Parameters
        ----------
        row : Each line in CSV passed from csv_reader
            If CSV line starts with 'T', create ToDo instance, else
            if line starts with 'D', create Deadline instance.

        Raises
        ------
        IndexError
            Raises IndexError if unable to get indexes of row.

        Returns
        -------
        None.

        """
        try:
            if row[0] == 'T':
                items.append(td.ToDo(row[1], True if row[2] == 'True' else False))
            elif row[0] == 'D':
                items.append(dl.Deadline(row[1], True if row[2] == 'True' else False, row[3]))
        except IndexError:
            raise IndexError
        return
    
    def save_data(self, items):
        """
        Method to save data to external CSV specified in attribute.

        Returns
        -------
        None.

        """
        with open(self.filename, "w", newline='') as csvfile:
            output = csv.writer(csvfile)
            for item in items:
                if isinstance(item, dl.Deadline):          
                    output_to_file = ["D",item.description,item.is_done,item.by]
                else:
                    output_to_file = ["T",item.description,item.is_done]
                output.writerow(output_to_file)

# exceptions.py

In [3]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class ZeroInputError(Error):
    """Raised when the input is 0"""
    pass

class InvalidMassInputError(Error):
    """Raised when nothing was done in a mass action command"""
    pass

class InvalidDeadlineInput(Error):
    """Raised when 'by:' keyword is not entered. Also base class for other
    deadline exceptions'"""
    pass

class NoDueDateError(InvalidDeadlineInput):
    """Raised when the ONLY due date is blank"""
    pass

class NoDueNoTaskError(InvalidDeadlineInput):
    """Raised when the both due date AND task is blank"""
    pass

class NoTaskError(InvalidDeadlineInput):
    """Raised when the both due date AND task is blank"""
    pass

# testcase.py

In [ ]:
import unittest
import exceptions as ex
import taskmanager as tm     

# test class
class TestSearch(unittest.TestCase):
    
    test_task_manager = tm.TaskManager()

    def test_add_item(self):
        with self.assertRaises(IndexError):
            self.test_task_manager.add_item("todo")
        with self.assertRaises(IndexError):
            self.test_task_manager.add_item("todo    ")
        self.assertEqual(self.test_task_manager.add_item("todo foo bar"), "New item: 'foo bar' added")
        self.assertEqual(self.test_task_manager.add_item("todo 1234567"), "New item: '1234567' added")
        self.assertEqual(self.test_task_manager.add_item("todo !@#$%^&*"), "New item: '!@#$%^&*' added")
        self.assertEqual(self.test_task_manager.add_item("todo todo"), "New item: 'todo' added")
        
    def test_add_deadline_item(self):
        self.assertEqual(self.test_task_manager.add_deadline_item("deadline foo by: bar")
                          , "New item: 'foo' added. Deadline: 'bar'")
        self.assertEqual(self.test_task_manager.add_deadline_item("deadline 123 by: 567")
                          , "New item: '123' added. Deadline: '567'")
        self.assertEqual(self.test_task_manager.add_deadline_item("deadline deadline by: deadline")
                          , "New item: 'deadline' added. Deadline: 'deadline'")
        self.assertEqual(self.test_task_manager.add_deadline_item("deadline !@#$% by:       ((*&^))      ")
                          , "New item: '!@#$%' added. Deadline: '((*&^))'")
     
        with self.assertRaises(ex.InvalidDeadlineInput):
            self.test_task_manager.add_deadline_item("deadline")
        with self.assertRaises(ex.NoTaskError):
            self.test_task_manager.add_deadline_item("deadline by: foo")
        with self.assertRaises(ex.NoDueDateError):
            self.test_task_manager.add_deadline_item("deadline foo by: ")
        with self.assertRaises(ex.NoDueNoTaskError):
            self.test_task_manager.add_deadline_item("deadline by: ")
        with self.assertRaises(ex.NoTaskError):
            self.test_task_manager.add_deadline_item("deadline by: by:")
        with self.assertRaises(ex.InvalidDeadlineInput):
            self.test_task_manager.add_deadline_item("deadline foo by")
        
# activate the test runner
if __name__ == '__main__':
    unittest.main()    